Импорт библиотек

In [56]:
import requests
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score, roc_curve, precision_recall_curve

Функция для обработки запроса на сервер

In [57]:
def send_json(body):
    myurl = 'http://127.0.0.1:8082/' + '/predict'
    headers = {'content-type': 'application/json; charset=utf-8'}
    response = requests.post(myurl, json=body, headers=headers)
    return response.json()['predictions']

Читаем тестовый датасет

In [58]:
test_df = pd.read_csv('X_test.csv', index_col='id')
test_df.head(5)

,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active
id,,,,,,,,,,,
66728,21770,1,156,64.0,140,80,2,1,0,0,1
69098,21876,1,170,85.0,160,90,1,1,0,0,1
59185,23270,1,151,90.0,130,80,1,1,0,0,1
49288,19741,1,159,97.0,120,80,1,1,0,0,1
62481,18395,1,164,68.0,120,80,1,1,0,0,1


Делаем пробный запрос

In [59]:
data = test_df.iloc[[0]].to_dict(orient='list')

In [60]:
response = send_json(data)
response

0.8089820827029013

Сделаем n запросов на сервер по тестовым данным и посмотрим метрики

In [61]:
test_target = pd.read_csv('y_test.csv', index_col='id')

In [62]:
n = 50
m_row_data = test_df.iloc[0:n]
m_row_target = test_target.iloc[0:n]
predictions = []
for i in range(n):
    r_data = m_row_data.iloc[[i]].to_dict(orient='list')
    r_response = send_json(r_data)
    predictions.append(r_response)

In [63]:
precision, recall, thresholds = precision_recall_curve(m_row_target, predictions)
fscore = (2 * precision * recall) / (precision + recall)
ix = np.argmax(fscore)
print(f'Best Threshold={thresholds[ix]}, F-Score={fscore[ix]:.3f}, Precision={precision[ix]:.3f}, Recall={recall[ix]:.3f}')

Best Threshold=0.5917157753383866, F-Score=0.833, Precision=0.909, Recall=0.769


In [64]:
roc_auc_score(y_score=predictions, y_true=m_row_target)

0.8958333333333333